# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fd3b0c30d30>
├── 'a' --> tensor([[-0.1512,  0.8631, -0.9579],
│                   [ 0.2685,  1.4242,  1.8872]])
└── 'x' --> <FastTreeValue 0x7fd3b0c30eb0>
    └── 'c' --> tensor([[-0.7394, -0.9615,  0.0496, -0.3056],
                        [-1.0702,  0.5972,  0.2575, -0.2341],
                        [-0.7285,  0.8987, -2.3792, -0.1314]])

In [4]:
t.a

tensor([[-0.1512,  0.8631, -0.9579],
        [ 0.2685,  1.4242,  1.8872]])

In [5]:
%timeit t.a

90.4 ns ± 0.167 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fd3b0c30d30>
├── 'a' --> tensor([[-0.9738, -2.0829,  0.4015],
│                   [ 0.9364, -0.0883, -0.4703]])
└── 'x' --> <FastTreeValue 0x7fd3b0c30eb0>
    └── 'c' --> tensor([[-0.7394, -0.9615,  0.0496, -0.3056],
                        [-1.0702,  0.5972,  0.2575, -0.2341],
                        [-0.7285,  0.8987, -2.3792, -0.1314]])

In [7]:
%timeit t.a = new_value

102 ns ± 0.147 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.1512,  0.8631, -0.9579],
               [ 0.2685,  1.4242,  1.8872]]),
    x: Batch(
           c: tensor([[-0.7394, -0.9615,  0.0496, -0.3056],
                      [-1.0702,  0.5972,  0.2575, -0.2341],
                      [-0.7285,  0.8987, -2.3792, -0.1314]]),
       ),
)

In [10]:
b.a

tensor([[-0.1512,  0.8631, -0.9579],
        [ 0.2685,  1.4242,  1.8872]])

In [11]:
%timeit b.a

85.5 ns ± 0.0902 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.6766,  1.4386,  0.3591],
               [-0.0959,  0.6404,  0.1968]]),
    x: Batch(
           c: tensor([[-0.7394, -0.9615,  0.0496, -0.3056],
                      [-1.0702,  0.5972,  0.2575, -0.2341],
                      [-0.7285,  0.8987, -2.3792, -0.1314]]),
       ),
)

In [13]:
%timeit b.a = new_value

671 ns ± 2.19 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.1 µs ± 2.23 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.5 µs ± 34.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

180 µs ± 16.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

177 µs ± 1.36 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fd2d0ca47c0>
├── 'a' --> tensor([[[-0.1512,  0.8631, -0.9579],
│                    [ 0.2685,  1.4242,  1.8872]],
│           
│                   [[-0.1512,  0.8631, -0.9579],
│                    [ 0.2685,  1.4242,  1.8872]],
│           
│                   [[-0.1512,  0.8631, -0.9579],
│                    [ 0.2685,  1.4242,  1.8872]],
│           
│                   [[-0.1512,  0.8631, -0.9579],
│                    [ 0.2685,  1.4242,  1.8872]],
│           
│                   [[-0.1512,  0.8631, -0.9579],
│                    [ 0.2685,  1.4242,  1.8872]],
│           
│                   [[-0.1512,  0.8631, -0.9579],
│                    [ 0.2685,  1.4242,  1.8872]],
│           
│                   [[-0.1512,  0.8631, -0.9579],
│                    [ 0.2685,  1.4242,  1.8872]],
│           
│                   [[-0.1512,  0.8631, -0.9579],
│                    [ 0.2685,  1.4242,  1.8872]]])
└── 'x' --> <FastTreeValue 0x7fd2d909a6a0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

40.7 µs ± 226 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fd2d0ca4040>
├── 'a' --> tensor([[-0.1512,  0.8631, -0.9579],
│                   [ 0.2685,  1.4242,  1.8872],
│                   [-0.1512,  0.8631, -0.9579],
│                   [ 0.2685,  1.4242,  1.8872],
│                   [-0.1512,  0.8631, -0.9579],
│                   [ 0.2685,  1.4242,  1.8872],
│                   [-0.1512,  0.8631, -0.9579],
│                   [ 0.2685,  1.4242,  1.8872],
│                   [-0.1512,  0.8631, -0.9579],
│                   [ 0.2685,  1.4242,  1.8872],
│                   [-0.1512,  0.8631, -0.9579],
│                   [ 0.2685,  1.4242,  1.8872],
│                   [-0.1512,  0.8631, -0.9579],
│                   [ 0.2685,  1.4242,  1.8872],
│                   [-0.1512,  0.8631, -0.9579],
│                   [ 0.2685,  1.4242,  1.8872]])
└── 'x' --> <FastTreeValue 0x7fd2cfc44f40>
    └── 'c' --> tensor([[-0.7394, -0.9615,  0.0496, -0.3056],
                        [-1.0702,  0.5972,  0.2575, -0.2341],
                 

In [23]:
%timeit t_cat(trees)

38.8 µs ± 580 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

75.3 µs ± 155 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.1512,  0.8631, -0.9579],
                [ 0.2685,  1.4242,  1.8872]],
       
               [[-0.1512,  0.8631, -0.9579],
                [ 0.2685,  1.4242,  1.8872]],
       
               [[-0.1512,  0.8631, -0.9579],
                [ 0.2685,  1.4242,  1.8872]],
       
               [[-0.1512,  0.8631, -0.9579],
                [ 0.2685,  1.4242,  1.8872]],
       
               [[-0.1512,  0.8631, -0.9579],
                [ 0.2685,  1.4242,  1.8872]],
       
               [[-0.1512,  0.8631, -0.9579],
                [ 0.2685,  1.4242,  1.8872]],
       
               [[-0.1512,  0.8631, -0.9579],
                [ 0.2685,  1.4242,  1.8872]],
       
               [[-0.1512,  0.8631, -0.9579],
                [ 0.2685,  1.4242,  1.8872]]]),
    x: Batch(
           c: tensor([[[-0.7394, -0.9615,  0.0496, -0.3056],
                       [-1.0702,  0.5972,  0.2575, -0.2341],
                       [-0.7285,  0.8987, -2.3792, -0.1314]],
         

In [26]:
%timeit Batch.stack(batches)

99.3 µs ± 398 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.1512,  0.8631, -0.9579],
               [ 0.2685,  1.4242,  1.8872],
               [-0.1512,  0.8631, -0.9579],
               [ 0.2685,  1.4242,  1.8872],
               [-0.1512,  0.8631, -0.9579],
               [ 0.2685,  1.4242,  1.8872],
               [-0.1512,  0.8631, -0.9579],
               [ 0.2685,  1.4242,  1.8872],
               [-0.1512,  0.8631, -0.9579],
               [ 0.2685,  1.4242,  1.8872],
               [-0.1512,  0.8631, -0.9579],
               [ 0.2685,  1.4242,  1.8872],
               [-0.1512,  0.8631, -0.9579],
               [ 0.2685,  1.4242,  1.8872],
               [-0.1512,  0.8631, -0.9579],
               [ 0.2685,  1.4242,  1.8872]]),
    x: Batch(
           c: tensor([[-0.7394, -0.9615,  0.0496, -0.3056],
                      [-1.0702,  0.5972,  0.2575, -0.2341],
                      [-0.7285,  0.8987, -2.3792, -0.1314],
                      [-0.7394, -0.9615,  0.0496, -0.3056],
                      [-1.0702,  

In [28]:
%timeit Batch.cat(batches)

181 µs ± 636 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

447 µs ± 5.45 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
